## Every Dream trainer

You will need your data prepared first before starting!  Don't waste rental fees if you're not ready to upload your files.  Your files should be captioned before you start with either the caption as the filename or in text files for each image alongside the image files.  See main README.md for more details. Tools are available to automatically caption your files.

[Instructions](https://github.com/victorchall/EveryDream-trainer/blob/main/README.md)

If you can sign up for Runpod here (shameless referral link): [Runpod](https://runpod.io?ref=oko38cd0)

If you are confused by the wall of text, join the discord here: [EveryDream Discord](https://discord.gg/uheqxU6sXN)

Make sure you have at least 40GB of Runpod **Volume** storage at a minimum so you don't waste training just 1 ckpt that is overtrained and have to start over.  Penny pinching on storage is ultimately a waste of your time and money!  This is setup to give you more than one ckpt so you don't overtrain.

### Starting model
Make sure you have your hugging face token ready to download the 1.5 mode. You can get one here: https://huggingface.co/settings/tokens
If you don't have a User Access Token, create one.  Or you can upload a starting checkpoint instead of using the HF download and skip that step, but you'll need to modify the starting model name when you start training (more info below).

In [ ]:
!git clone https://github.com/victorchall/everydream-trainer
%cd everydream-trainer

import codecs
finish_msg = codecs.encode('QBAR', 'rot_13')

print(finish_msg)

## Install dependencies

**This will take a couple minutes.  Wait until it says "DONE" to move on.** 
You can ignore "warnings."

In [ ]:
!pip install -q omegaconf
!pip install -q einops
!pip install -q pytorch-lightning==1.6.5
!pip install -q test-tube
!pip install -q transformers==4.19.2
!pip install -q kornia
!pip install -e git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers
!pip install -e git+https://github.com/openai/CLIP.git@main#egg=clip
!pip install -q setuptools==59.5.0
!pip install -q pillow==9.0.1
!pip install -q torchmetrics==0.6.0
!pip install -e .
!pip install huggingface_hub
!pip install ipywidgets==7.7.1
import ipywidgets as widgets
print(finish_msg)

## Now that dependencies are installed, ready to move on!

## Log into huggingface
Run the cell below and paste your token into the prompt.  You can get your token from your [huggingface account page](https://huggingface.co/settings/tokens).

The token will not show on the screen, just press enter after you paste it.

Then run the following cell to download the base checkpoint (may take a minute).

In [17]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from huggingface_hub import hf_hub_download
downloaded_model_path = hf_hub_download(
 repo_id="runwayml/stable-diffusion-v1-5",
 filename="v1-5-pruned-emaonly.ckpt",
 use_auth_token=True
)
print(downloaded_model_path) # cache location
print(finish_msg)

# Upload training files

Ues the navigation on the left to open **"every-dream-trainer"** folder, then open **"input"** and upload your training files using the **up arrow button** above the file explorer.  

You can upload your images while you wait for the base checkpoint to download above, but do not continue until the checkpoint download is done.

If your captions are in .txt files, upload those in the same folder as the images. 

![upload](./demo/runpodupload.png)

You can check there are files in the folder by running the cell below (optional, just prints first 10).

In [1]:
!ls -U input | head -10
# at least a few image filenames should show below, if not you uploaded to the wrong folder

'ls' is not recognized as an internal or external command,
operable program or batch file.


## Tweak your YAML
You can adjust the YAML file to change the training parameters.  The v1-fine yamls are in the everydream-trainer/configs/stable-diffusion folder.  By default the "v1-finetune_runpod.yaml" is used here with some sane defaults for one subject with 20-30 images.

You can also upload your own in that folder and use it below.

Instructions are here: [EveryDream README](https://github.com/victorchall/EveryDream-trainer/blob/main/README.md) (hopefully you already read this?)

[Runpod YAML](configs/stable-diffusion/v1-finetune_runpod.yaml) is a good starting point for small datasets (30-50 images) and **is the default in the command below**. It will only keep 2 checkpoints.

If you need help with larger training, join the Discord for advice on making a custom yaml.

In [ ]:
# change to use a custom yaml, but RUN THIS CELL NO MATTER WHAT to set your yaml path
my_yaml = "configs/stable-diffusion/v1-finetune_runpod.yaml"
# downloaded_model_path = "v1-5-pruned-emaonly.ckpt"  # this is the default, but you can change it by uncommenting if you uploaded a file into /everydream-trainer
print(downloaded_model_path) # reminder in case something went wrong with download
print(f"yaml set to: {my_yaml}")

## Run the trainer
Next cell runs training.  This will take a while depending on your number of images, repeats, and max_epochs.  

You can watch for test images in the logs folder.

In [ ]:
# run the trainer, wait until it finishes then SCROLL DOWN to the next cell
!python main.py --base {my_yaml} -t  --actual_resume {downloaded_model_path} -n test --data_root input
print(finish_msg)

## Prune your checkpoints
This will create 2GB pruned files for all your checkpoints and delete the 11GB files.

In [ ]:
# prune the ckpts
!python scripts/autoprune_all.py --delete

## Download your checkpoints

Use the file explorer on the left, go into the "every-dream-trainer" folder.

Look for all the ckpt files that say "-pruned" on the end.  Download them and you're done!  Test each one in your favorite webui.

[EveryDream Discord](https://discord.gg/uheqxU6sXN)